In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade transformers
!pip install --upgrade datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load dataset
file_path = '/content/drive/MyDrive/ML_project/ML_project/merged_data.csv'
df = pd.read_csv(file_path)

# Rename columns for HuggingFace compatibility
df = df.rename(columns={'sentence': 'incorrect_sentence', 'corrections': 'correct_sentence'})

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained('prithivida/grammar_error_correcter_v1')

# Tokenize Dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['incorrect_sentence'], padding='max_length', truncation=True, max_length=64)
    targets = tokenizer(examples['correct_sentence'], padding='max_length', truncation=True, max_length=64)
    inputs['labels'] = targets['input_ids']
    return inputs


tokenized_dataset = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Map:   0%|          | 0/2772 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# Load Pre-trained GECToR Model
model = AutoModelForSeq2SeqLM.from_pretrained('prithivida/grammar_error_correcter_v1')


training_args = TrainingArguments(
    output_dir="./gec_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    logging_dir="./logs",
    gradient_accumulation_steps=4,
    fp16=True  # Enable mixed precision training
)



# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Start Training
trainer.train()

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-918195de50f9>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.079297
2,0.319900,0.059259


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1038, training_loss=2.124557647631577, metrics={'train_runtime': 971.791, 'train_samples_per_second': 8.557, 'train_steps_per_second': 1.068, 'total_flos': 631489332510720.0, 'train_loss': 2.124557647631577, 'epoch': 2.9927849927849928})

In [ ]:
import torch

# Ensure the model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Correct Sentence Function
def correct_sentence(sentence):
    # Move the inputs to the same device as the model
    inputs = tokenizer(sentence, return_tensors='pt', max_length=64, truncation=True).to(device)

    # Generate the corrected sentence
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model.generate(**inputs)

    # Decode the generated sentence
    corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_sentence

# Example
test_sentence = "I is happy to saw you."
corrected = correct_sentence(test_sentence)
print("Original Sentence:", test_sentence)
print("Corrected Sentence:", corrected)


Original Sentence: I is happy to saw you.
Corrected Sentence: I am happy to see you.


In [ ]:
model.save_pretrained('/content/drive/MyDrive/fine_tuned_gec_model')
tokenizer.save_pretrained('/content/drive/MyDrive/fine_tuned_gec_model')


('/content/drive/MyDrive/fine_tuned_gec_model/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_gec_model/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_gec_model/spiece.model',
 '/content/drive/MyDrive/fine_tuned_gec_model/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_gec_model/tokenizer.json')

In [ ]:
test_sentence = "I don’t need no help from nobody."
corrected = correct_sentence(test_sentence)
print("Original Sentence:", test_sentence)
print("Corrected Sentence:", corrected)

Original Sentence: I don’t need no help from nobody.
Corrected Sentence: I don’t need any help from anybody.


In [ ]:
test_sentence = "Walking through the park, the flowers were blooming beautifully."
corrected = correct_sentence(test_sentence)
print("Original Sentence:", test_sentence)
print("Corrected Sentence:", corrected)

Original Sentence: Walking through the park, the flowers were blooming beautifully.
Corrected Sentence: Walking through the park, the flowers were blooming beautifully.


In [ ]:
test_sentence = "She will goes to the market yesterday but forgot her wallet."
corrected = correct_sentence(test_sentence)
print("Original Sentence:", test_sentence)
print("Corrected Sentence:", corrected)

Original Sentence: She will goes to the market yesterday but forgot her wallet.
Corrected Sentence: She went to the market yesterday but forgot her wallet.


In [ ]:
# Test Sentences
test_sentences = [
    "The group of students was late because they don't knew the meeting time was change.",
    "She will goes to the market yesterday but forgot her wallet.",
    "Running to catch the bus, the backpack fell off his shoulder.",
    "Walking through the park, the flowers were blooming beautifully.",
    "I don’t need no help from nobody.",
    "Me and him went to the store, and he bought candy for they.",
    "She is good in cooking, but bad at swimming.",
    "Although he was tired. Decided to keep working.",
    "I went to the store I bought some bread it was very crowded.",
    "Their going too the party two, to."
]

# Test the Model
for sentence in test_sentences:
    corrected = correct_sentence(sentence)
    print(f"Original: {sentence}")
    print(f"Corrected: {corrected}\n")


Original: The group of students was late because they don't knew the meeting time was change.
Corrected: The group of students were late because they didn't know the meeting time was changing.

Original: She will goes to the market yesterday but forgot her wallet.
Corrected: She went to the market yesterday but forgot her wallet.

Original: Running to catch the bus, the backpack fell off his shoulder.
Corrected: Running to catch the bus, his backpack fell off his shoulder.

Original: Walking through the park, the flowers were blooming beautifully.
Corrected: Walking through the park, the flowers were blooming beautifully.

Original: I don’t need no help from nobody.
Corrected: I don’t need any help from anybody.

Original: Me and him went to the store, and he bought candy for they.
Corrected: Me and him went to the store, and he bought candy for them.

Original: She is good in cooking, but bad at swimming.
Corrected: She is good at cooking, but bad at swimming.

Original: Although he w

In [ ]:
# Expanded Quick Accuracy Check Sentences
test_cases = [
    ("She will goes to the market yesterday but forgot her wallet.", "She went to the market yesterday but forgot her wallet."),
    ("Me and him goes to store and buyed candies.", "He and I went to the store and bought candies."),
    ("He don't knows nothing about fixing cars.", "He doesn't know anything about fixing cars."),
    ("Although he was tired. Decided to keep working.", "Although he was tired, he decided to keep working."),
    ("Their going too the park later, isn't it?", "They're going to the park later, aren't they?"),
    ("The dog bark loudly every morning.", "The dog barks loudly every morning."),
    ("He didn't knew about the meeting.", "He didn't know about the meeting."),
    ("Me and her went to the cinema.", "She and I went to the cinema."),
    ("She don't wants to go out tonight.", "She doesn't want to go out tonight."),
    ("We was watching a movie yesterday.", "We were watching a movie yesterday."),
    ("I didn't see nobody at the party.", "I didn't see anybody at the party."),
    ("She is good in math.", "She is good at math."),
    ("The flowers smells nice.", "The flowers smell nice."),
    ("Running fast, the finish line was reached.", "Running fast, he reached the finish line."),
    ("She buyed a new dress last week.", "She bought a new dress last week."),
    ("Them is going to the event.", "They are going to the event."),
    ("He was so tired, he slept all the day.", "He was so tired that he slept all day."),
    ("I seen him at the mall yesterday.", "I saw him at the mall yesterday."),
    ("She enjoy to read books.", "She enjoys reading books."),
    ("Her don't like spicy food.", "She doesn't like spicy food."),
    ("They was playing soccer when I arrived.", "They were playing soccer when I arrived."),
    ("Him is taller than me.", "He is taller than me."),
    ("The book were on the table.", "The book was on the table."),
    ("He don't have no money.", "He doesn't have any money."),
    ("The cat was sitting on a tree who was very high.", "The cat was sitting on a tree that was very high."),
    ("She is married with a lawyer.", "She is married to a lawyer."),
    ("He doesn't knows where the library is.", "He doesn't know where the library is."),
    ("While walking through the park, the birds sang beautifully.", "While walking through the park, he heard the birds singing beautifully."),
    ("Me and my friend likes pizza.", "My friend and I like pizza."),
    ("He was interested on the new project.", "He was interested in the new project."),
    ("I cannot able to do it.", "I am not able to do it."),
    ("She is knowing the answer.", "She knows the answer."),
    ("If I was you, I would apologize.", "If I were you, I would apologize."),
    ("The movie was too much long.", "The movie was too long."),
    ("I am agree with your opinion.", "I agree with your opinion."),
    ("She said me to come early.", "She told me to come early."),
    ("The train leave at 5 PM every day.", "The train leaves at 5 PM every day."),
    ("I have seen her yesterday.", "I saw her yesterday."),
    ("He suggested me to join the team.", "He suggested that I join the team."),
    ("They was not ready for the trip.", "They were not ready for the trip."),
    ("Neither of the boys are responsible.", "Neither of the boys is responsible."),
    ("She sings good.", "She sings well."),
    ("He is very interesting in science.", "He is very interested in science."),
    ("Me went to the party alone.", "I went to the party alone."),
    ("He didn't knew what to say.", "He didn't know what to say."),
    ("The team are playing well.", "The team is playing well."),
    ("You should to try harder.", "You should try harder."),
    ("He is more taller than me.", "He is taller than me."),
    ("She do her homework every evening.", "She does her homework every evening."),
    ("Each of the students were given a book.", "Each of the students was given a book."),
    ("He doesn't likes coffee.", "He doesn't like coffee."),
    ("This place is more better than the other one.", "This place is better than the other one."),
    ("I wish I was there.", "I wish I were there."),
    ("He has went to the market.", "He has gone to the market."),
    ("The dog is barking on the stranger.", "The dog is barking at the stranger."),
    ("He explained me the problem.", "He explained the problem to me."),
    ("She hasn't no time to talk.", "She doesn't have time to talk."),
    ("I am looking forward to meet you.", "I am looking forward to meeting you."),
    ("She can sings beautifully.", "She can sing beautifully."),
    ("He don't likes working on weekends.", "He doesn't like working on weekends."),
    ("They didn't had lunch yet.", "They haven't had lunch yet."),
    ("He is very tired, but he keep working.", "He is very tired, but he keeps working."),
    ("This is the house where I was born in.", "This is the house where I was born."),
    ("He suggested me that I should join.", "He suggested that I should join."),
    ("She do not wants to leave.", "She does not want to leave.")
]

# Test the model
correct_predictions = 0
incorrect_predictions = []

for incorrect, expected in test_cases:
    corrected = correct_sentence(incorrect)
    print(f"Original: {incorrect}")
    print(f"Expected: {expected}")
    print(f"Corrected: {corrected}\n")

    if corrected.strip() == expected.strip():
        correct_predictions += 1
    else:
        incorrect_predictions.append((incorrect, expected, corrected))

# Calculate Quick Accuracy
accuracy = (correct_predictions / len(test_cases)) * 100

# Display Results
print(f"\n✅ Quick Estimated Accuracy: {accuracy:.2f}%")
print(f"❌ Number of Incorrect Predictions: {len(incorrect_predictions)}\n")

# Display Incorrect Predictions
if incorrect_predictions:
    print("🚨 **Incorrect Predictions:**")
    for i, (incorrect, expected, corrected) in enumerate(incorrect_predictions, 1):
        print(f"{i}. Original: {incorrect}")
        print(f"   Expected: {expected}")
        print(f"   Corrected: {corrected}\n")


Original: She will goes to the market yesterday but forgot her wallet.
Expected: She went to the market yesterday but forgot her wallet.
Corrected: She went to the market yesterday but forgot her wallet.

Original: Me and him goes to store and buyed candies.
Expected: He and I went to the store and bought candies.
Corrected: Me and him went to the store and bought candies.

Original: He don't knows nothing about fixing cars.
Expected: He doesn't know anything about fixing cars.
Corrected: He doesn't know anything about fixing cars.

Original: Although he was tired. Decided to keep working.
Expected: Although he was tired, he decided to keep working.
Corrected: Although he was tired he decided to keep working.

Original: Their going too the park later, isn't it?
Expected: They're going to the park later, aren't they?
Corrected: They're going to the park later, isn't it?

Original: The dog bark loudly every morning.
Expected: The dog barks loudly every morning.
Corrected: The dog barks l

In [ ]:
print(len(test_cases))

65
